<a href="https://colab.research.google.com/github/orz-c3/-/blob/main/111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import rpy2.robjects as robjects
import pandas as pd
import numpy as np

# 加载 .RData 文件
robjects.r['load']('riboflavin.RData')

# 列出当前环境中的所有对象
objects = robjects.r['ls']()
print("R环境中的对象:", objects)

# 定义要访问的对象
obj_names = ['X', 'Y']
data_frames = {}

for obj_name in obj_names:
    data = robjects.r[obj_name]

    # 检查对象的类型
    data_class = robjects.r['class'](data)
    print(f"对象 '{obj_name}' 的类型:", data_class)

    # 根据对象类型进行处理，并转换为 DataFrame
    if data_class[0] == 'data.frame':
        df = pd.DataFrame(data)
    elif data_class[0] == 'matrix':
        df = pd.DataFrame(data)
    elif data_class[0] == 'list':
        # 将列表转换为 DataFrame
        df = pd.DataFrame({f"col_{i}": data[i] for i in range(len(data))})
    elif data_class[0] in ['numeric', 'integer']:
        # 向量转换为 DataFrame
        df = pd.DataFrame(data)
    elif data_class[0] == 'array':
        # 将 array 转换为 DataFrame
        # 这里我们假设 array 是二维的
        df = pd.DataFrame(np.array(data))
    else:
        print(f"对象 '{obj_name}' 的类型 '{data_class[0]}' 不被支持或未处理。")
        continue

    # 存储数据框
    data_frames[obj_name] = df

    # 导出为 CSV 文件
    df.to_csv(f"{obj_name}.csv", index=False)
    print(f"对象 '{obj_name}' 已导出为 '{obj_name}.csv'。")

# 如果需要，也可以导出为 Excel 文件
# with pd.ExcelWriter('data.xlsx') as writer:
#     for name, df in data_frames.items():
#         df.to_excel(writer, sheet_name=name, index=False)
#         print(f"对象 '{name}' 已导出为 Excel 文件。")


R环境中的对象: [1] "X" "Y"

对象 'X' 的类型: [1] "matrix" "array" 

对象 'X' 已导出为 'X.csv'。
对象 'Y' 的类型: [1] "array"

对象 'Y' 已导出为 'Y.csv'。


In [16]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

# 导入数据
X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')

# 确保 Y 是一维的
Y = Y.values.ravel()

# 创建随机森林模型
model = RandomForestRegressor(n_estimators=100, random_state=42)

# 训练模型
model.fit(X, Y)

# 获取特征的重要性
importances = model.feature_importances_

# 将特征重要性与特征名称结合
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# 按照重要性排序
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# 选出前5个重要特征
top_features = feature_importance.head(5)

print("与 Y 最相关的前 5 个特征:")
print(top_features)


与 Y 最相关的前 5 个特征:
     Feature  Importance
4002    4002    0.076301
2383    2383    0.049331
439      439    0.047942
2554    2554    0.039340
1122    1122    0.026610
